<a href="https://colab.research.google.com/github/ireneher11/Math-120-Final-Project/blob/main/Math120_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Book Dataset Analysis and Book Genre Predictor
This project is about analyzing book datasets. There are 3 questions that this notebook will be focusing on:
1. What is the relationship between the numbers of sales of a book and its genre?
2. Do top selling authors tend to publish many books?
3. Is it possible to predict the genre of a book based on its title and summary? How accurate is it?

## Environment Setup

In [1]:
import os
import sys

# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("Running in Google Colab")

    # Clone repository if in Colab
    if not os.path.exists('/content/math120_final_project/'):
        !git clone https://github.com/ireneher11/Math-120-Final-Project.git

    # Change to project directory
    os.chdir('/content/Math-120-Final-Project')

except ImportError:
    IN_COLAB = False
    print("Running locally")

# Add src directory to Python path
if 'src' not in sys.path:
    sys.path.append('src')

print(f"Current working directory: {os.getcwd()}")

Running in Google Colab
Cloning into 'Math-120-Final-Project'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 48 (delta 7), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (48/48), 8.66 MiB | 3.81 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Current working directory: /content/Math-120-Final-Project


## Importing and Cleaning the datasets


In [2]:
#import libraries
import numpy as np
import pandas as pd
import plotly.express as px

In [3]:
#first dataset
df = pd.read_csv('data_raw/Book_Details.csv')
new_df = df[["book_id", "book_title", "book_details", "author", "genres", "num_reviews", "average_rating"]].copy()

#checking for missing values and duplicates
new_df.isna().sum()
new_df.replace(["", " ", "[None]", "[]"], np.nan, inplace=True)

new_df.duplicated().sum()
new_df[new_df.duplicated(keep=False)]
clean_df = new_df.drop_duplicates()

clean_df.head()


,book_id,book_title,book_details,author,genres,num_reviews,average_rating
0,1,Harry Potter and the Half-Blood Prince,"It is the middle of the summer, but there is a...",J.K. Rowling,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",58398,4.58
1,2,Harry Potter and the Order of the Phoenix,Harry Potter is about to start his fifth year ...,J.K. Rowling,"['Young Adult', 'Fiction', 'Magic', 'Childrens...",64300,4.50
2,3,Harry Potter and the Sorcerer's Stone,Harry Potter has no idea how famous he is. Tha...,J.K. Rowling,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",163493,4.47
3,5,Harry Potter and the Prisoner of Azkaban,"Harry Potter, along with his best friends, Ron...",J.K. Rowling,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",84959,4.58
4,6,Harry Potter and the Goblet of Fire,It is the summer holidays and soon Harry Potte...,J.K. Rowling,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...",69961,4.57


In [4]:
#second dataset
df1 = pd.read_csv('data_raw/Books_Data.csv')

df1.head()
df1_new = df1[["Publishing Year", "Book Name", "Author", "language_code", "gross sales", "publisher revenue", "sale price"]].copy()
df1_new.head()

#checking for missing values and duplicates
df1_new.isna().sum()

#drop blank book names
df1_new[df1_new["Book Name"].isna()]
new_df1 = df1_new.dropna(subset=["Book Name"]).copy()
new_df1.isna().sum()

new_df1.duplicated().sum()

#turn publising year into int
new_df1["Publishing Year"] = new_df1["Publishing Year"].astype(int)
new_df1.head()

,Publishing Year,Book Name,Author,language_code,gross sales,publisher revenue,sale price
0,1975,Beowulf,"Unknown, Seamus Heaney",en-US,34160.0,20496.0,4.88
1,1987,Batman: Year One,"Frank Miller, David Mazzucchelli, Richmond Lew...",eng,12437.5,7462.5,1.99
2,2015,Go Set a Watchman,Harper Lee,eng,47795.0,28677.0,8.69
3,2008,When You Are Engulfed in Flames,David Sedaris,en-US,41250.0,24750.0,7.50
4,2011,Daughter of Smoke & Bone,Laini Taylor,eng,37952.5,22771.5,7.99


In [5]:
#merging the data
books_data = pd.merge(clean_df, new_df1, left_on="book_title", right_on="Book Name", how="inner")

books_data["Book Title"] = books_data["book_title"].combine_first(books_data["Book Name"])
books_data = books_data.drop(columns=["book_title", "Book Name"])

#organizing and cleaning new dataset
books_data = books_data[["book_id", "Book Title", "author", "genres", "book_details", "num_reviews", "average_rating", "Publishing Year", "language_code", "gross sales", "publisher revenue"]].copy()

#renaming columns
books_renamed = books_data.rename(columns={
    "book_id": "Book ID",
    "Book Title": "Book Title",
    "author": "Author",
    "book_details": "Summary",
    "num_reviews": "Number of Reviews",
    "average_rating": "Average Rating",
    "Publishing Year": "Publishing Year",
    "language_code": "Language Code",
    "gross sales": "Gross Sales",
    "publisher revenue": "Publisher Revenue"
})

#check for missing values and duplicates
books_renamed.isna().sum()
books_renamed.duplicated().sum()

#assigning only one genre in Genre column
#check the type
books_renamed['genres'].dtype
type(books_renamed['genres'][0])

books_renamed1 = books_renamed.copy()
books_renamed1['genres'] = books_renamed['genres'].apply(lambda x: x.split(",")[0][2:-1] if pd.notna(x) else None)

books_df = books_renamed1.rename(columns={"genres": "Genre"})
books_df.head()
#books_df['Genre'].value_counts()


,Book ID,Book Title,Author,Genre,Summary,Number of Reviews,Average Rating,Publishing Year,Language Code,Gross Sales,Publisher Revenue
0,28,Notes from a Small Island,Bill Bryson,Travel,"""Suddenly, in the space of a moment, I realize...",5231,3.91,1995,eng,442.89,0.000
1,93,Heidi,Johanna Spyri,Classics,Little orphan Heidi goes to live high in the A...,6263,4.03,1880,eng,3730.65,2238.390
2,112,Children of Dune,Frank Herbert,Science Fiction,The desert planet of Arrakis has begun to grow...,8379,3.96,1976,eng,115.83,0.000
3,122,The Power of One,Bryce Courtenay,Fiction,"In 1939, as Hitler casts his enormous, cruel s...",5930,4.35,1989,eng,846.94,508.164
4,365,Dirk Gently's Holistic Detective Agency,Douglas Adams,Science Fiction,"What do a dead cat, a computer whiz-kid, an El...",4505,3.97,1987,en-US,478.40,0.000


In [6]:
#save cleaned data as csv
os.makedirs(os.path.dirname('data/processed_book_data.csv'), exist_ok=True)
books_df.to_csv('data/processed_book_data.csv', index=False)

##Visualization

##2. What is the relationship between the numbers of sales of a book and its genre?

In [7]:
genre_sales = books_df.groupby('Genre')['Gross Sales'].sum().reset_index()

In [8]:
#pie chart
fig = px.pie(genre_sales, values='Gross Sales', names='Genre', title='Distribution of Gross Sales by Genre')
fig.show()

In [9]:
#bar chart
barchart = px.bar(genre_sales, x='Gross Sales', y='Genre', orientation='h', title='Distribution of Gross Sales by Genre')
barchart.update_layout(width=900, height=800)
barchart.show()

Based on the charts, Fiction is the most successfully. Fantasy ranks second, considering that it is a subset of fiction, it did very well.

## 2. Do top selling authors tend to publish many books?

In [10]:
author_sales = books_df.groupby('Author')['Gross Sales'].sum().reset_index()
top_authors = author_sales.nlargest(10, 'Gross Sales')

df_top_authors = books_df[books_df['Author'].isin(top_authors['Author'])]
fig4 = px.bar(df_top_authors, x='Gross Sales', y='Author', color="Book Title", orientation='h', title='Top 10 Authors by Gross Sales')
fig4.update_traces(
    hoverlabel=dict(
        font_color="white"
    )
)
fig4.show()

As seen in the bar chart, most of the top selling authors do really well on one book and not as good on their others. On the other hand, Stephen King has many published books and the sales are distributed differently than the other authors.

## 3. Is it possible to predict the genre of a book based on its title and summary? How accurate is it?

Bayes' Theorem for book genre prediction:

$$
P(\text{Genre} \mid \text{Title}) = \frac{P(\text{Title} \mid \text{Genre}) \cdot P(\text{Genre})}{P(\text{Title})}
$$

In [11]:
#import libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
#Genres with less than 10 count ends up in Other category
genre_counts = books_df['Genre'].value_counts()
books = books_df.copy()
books['Genre'] = books_df["Genre"].apply(lambda x: x if genre_counts[x] > 10 else 'Others')
print(books['Genre'].value_counts())

Genre
Fiction               191
Fantasy               159
Others                 94
Young Adult            72
Nonfiction             54
Horror                 52
Romance                48
Classics               44
Mystery                39
Historical Fiction     36
Science Fiction        35
Chick Lit              12
Picture Books          12
Name: count, dtype: int64


In [13]:
# predictor class
class BookGenrePredictor:
  """
  Predicts the genre of a book based on its title or summary.
  """
  #things to initialize when using the book genre predictor
  def __init__(self):
      self.vectorizer = CountVectorizer(
          analyzer=lambda x:x,
          token_pattern = None
      ) #for Bags of words
      self.model = MultinomialNB() #Naive Bayes model
      self.label_encoder = LabelEncoder() #Encoding genres
      self.genre_counts = None #genre frequencies
      self.stopwords = set(stopwords.words('english')) #stopwords
      self.genres = books['Genre'].tolist()
      self.text_column = None

  #tokenize the data
  def tokenize_data(self, texts):
    """
    Tokenizes the input text by removing stopwords and punctuation along
    with converting text to lowercase.
    """
    if isinstance(texts, str):
      texts = [texts] #convert single string to list
    elif not isinstance(texts, list):
      raise ValueError("Input must be a string or a list of strings.")

    #clean text and remove stopwords
    all_tokens = []
    for text in texts:
      clean_text = re.sub(r'[^\w\s]', '', text.lower()).split()
      tokens = [word for word in clean_text if word not in self.stopwords]
      all_tokens.append(tokens)
    return all_tokens

  def train(self, df, text_column, genre_column='Genre', test_size=0.2, random_state=42):
    """
    Trains the book genre predictor model using the provided dataset.
    """
    if text_column not in df.columns:
      raise ValueError(f"Column '{text_column}' not found in the dataset.")
    if genre_column not in df.columns:
      raise ValueError(f"Column '{genre_column}' not found in the dataset.")
    self.text_column = text_column

    #tokenize text
    X_tokens = self.tokenize_data(df[text_column].tolist())

    #encoding genre labels
    y = self.label_encoder.fit_transform(df[genre_column])

    #vectorize(bags of words) text
    X = self.vectorizer.fit_transform(X_tokens)

    #split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    #Train using naive bayes
    self.model.fit(X_train, y_train)

    y_pred = self.model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy}")
    print(classification_report(y_test, y_pred, target_names=self.label_encoder.classes_, zero_division=0))
    return self

  def prediction_table(self, texts, true_genres):
    """
    Generates a prediction table for the provided texts and true genres.
    """
    if not isinstance(texts, list) or not isinstance(true_genres, list):
      raise ValueError("Input must be a list of strings.")
    if len(texts) != len(true_genres):
      raise ValueError("Input lists must have the same length")

    y_pred = self.predict(texts)

    df = pd.DataFrame({
        'Text': texts,
        'Predicted Genre': y_pred,
        'Actual Genre': true_genres
    })
    return df

  def predict(self, text):
    """
    Predicts the genre of a book based on its title or summary.
    """
    if self.text_column is None:
      raise ValueError("Model not trained. Call 'train' method first.")

    X_tokens = self.tokenize_data(text)
    X = self.vectorizer.transform(X_tokens)
    y_pred = self.model.predict(X)
    genre = self.label_encoder.inverse_transform(y_pred)[0]
    return f"The predicted genre is: {genre}"


In [14]:
#Predicting based on book title
genres = books['Genre'].tolist()
titles = books['Book Title'].tolist()

pred1 = BookGenrePredictor()
#pred1.tokenize_data(titles)
pred1.train(books, text_column='Book Title', genre_column='Genre')

df_pred1 = pred1.prediction_table(titles, genres)
df_pred1.head()

Accuracy: 0.32941176470588235
                    precision    recall  f1-score   support

         Chick Lit       0.00      0.00      0.00         2
          Classics       0.67      0.22      0.33         9
           Fantasy       0.53      0.50      0.52        32
           Fiction       0.28      0.79      0.41        38
Historical Fiction       0.00      0.00      0.00         7
            Horror       0.00      0.00      0.00        10
           Mystery       1.00      0.25      0.40         8
        Nonfiction       0.00      0.00      0.00        11
            Others       0.21      0.21      0.21        19
     Picture Books       0.00      0.00      0.00         2
           Romance       0.00      0.00      0.00        10
   Science Fiction       1.00      0.29      0.44         7
       Young Adult       0.00      0.00      0.00        15

          accuracy                           0.33       170
         macro avg       0.28      0.17      0.18       170
      we

,Text,Predicted Genre,Actual Genre
0,Notes from a Small Island,The predicted genre is: Others,Others
1,Heidi,The predicted genre is: Others,Classics
2,Children of Dune,The predicted genre is: Others,Science Fiction
3,The Power of One,The predicted genre is: Others,Fiction
4,Dirk Gently's Holistic Detective Agency,The predicted genre is: Others,Science Fiction


In [15]:
pred1.predict("Dune")
pred1.predict("Harry Potter and the Cursed Child")

'The predicted genre is: Fiction'

In [16]:
#Predicting based on book summary
summaries = books['Summary'].tolist()

pred2 = BookGenrePredictor()
pred2.tokenize_data(titles)
pred2.train(books, text_column='Summary', genre_column='Genre')

df_pred2 = pred2.prediction_table(summaries, genres)
df_pred2.head()

Accuracy: 0.45294117647058824
                    precision    recall  f1-score   support

         Chick Lit       0.00      0.00      0.00         2
          Classics       1.00      0.33      0.50         9
           Fantasy       0.59      0.84      0.69        32
           Fiction       0.32      0.84      0.46        38
Historical Fiction       0.00      0.00      0.00         7
            Horror       0.00      0.00      0.00        10
           Mystery       1.00      0.38      0.55         8
        Nonfiction       0.25      0.09      0.13        11
            Others       0.57      0.21      0.31        19
     Picture Books       0.00      0.00      0.00         2
           Romance       0.00      0.00      0.00        10
   Science Fiction       1.00      0.29      0.44         7
       Young Adult       1.00      0.33      0.50        15

          accuracy                           0.45       170
         macro avg       0.44      0.25      0.28       170
      we

,Text,Predicted Genre,Actual Genre
0,"""Suddenly, in the space of a moment, I realize...",The predicted genre is: Others,Others
1,Little orphan Heidi goes to live high in the A...,The predicted genre is: Others,Classics
2,The desert planet of Arrakis has begun to grow...,The predicted genre is: Others,Science Fiction
3,"In 1939, as Hitler casts his enormous, cruel s...",The predicted genre is: Others,Fiction
4,"What do a dead cat, a computer whiz-kid, an El...",The predicted genre is: Others,Science Fiction


In [18]:
pred2.predict("Set on the desert planet Arrakis, Dune is the story of the boy Paul Atreides, heir to a noble family tasked with ruling an inhospitable world where the only thing of value is the “spice” melange, a drug capable of extending life and enhancing consciousness. Coveted across the known universe, melange is a prize worth killing for...When House Atreides is betrayed, the destruction of Paul’s family will set the boy on a journey toward a destiny greater than he could ever have imagined. And as he evolves into the mysterious man known as Muad’Dib, he will bring to fruition humankind’s most ancient and unattainable dream.")
pred2.predict("It was always difficult being Harry Potter, and it isn't much easier now that he is an overworked employee of the Ministry of Magic, a husband, and a father of three school-age children.While Harry grapples with a past that refuses to stay where it belongs, his youngest son, Albus, must struggle with the weight of a family legacy he never wanted. As past and present fuse ominously, both father and son learn the uncomfortable truth: sometimes, darkness comes from unexpected places.")

'The predicted genre is: Fantasy'

We can try to predict the genre using the title but it will have a very low accurracy rate because there are very little words in a title. There's not enough information that can be drawn from the title. Using the summary is a bit better but it still has a low accurracy rate. This dataset is also small so there is not enough samples of each of the genre for the model to train on.

##Conclusion       



This project analyzed book datasets by demonstrating:
- Loading the data
- Augumenting the dataset and then merging them into one
- Using charts for data visualization
- Using classes and methods to build functions for predicting genres.